### BE18B006_BE18B011 Data Contest Notebook

In [39]:
#from google.colab import drive

#drive.mount('/content/gdrive')
#%cd /content/gdrive/MyDrive/Colab\ Notebooks/PRML

In [40]:
#importing libraries
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, classification_report, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.feature_selection import f_classif, SelectPercentile, chi2, SelectFromModel
from sklearn.preprocessing import StandardScaler

# FEATURE SELECTION

In [41]:
df1 = pd.read_csv('Dataset_1_Training.csv')
colnames = df1['ID_REF']
df1 = df1.transpose().reset_index().rename(columns = colnames)
df1 = df1.drop(index = 0)
df1 = df1.rename(columns={'index': 'ID_REF'})
df1['CO: 1'], df1['CO: 2'] = df1['CO: 1'].astype(int), df1['CO: 2'].astype(int)

X, y1, y2 = df1.iloc[:, 1:-2], df1.iloc[:, -2], df1.iloc[:, -1]

# Train test split
#df1 = df1.T.drop(['ID_REF'],axis=0)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size = 0.15, random_state = 30)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size = 0.15, random_state = 30)

In [42]:
df2 = pd.read_csv('Dataset_2_Training.csv')
colnames = df2['ID_REF']
df2 = df2.transpose().reset_index().rename(columns = colnames)
df2 = df2.drop(index = 0)
df2 = df2.rename(columns={'index': 'ID_REF'})
df2['CO: 3'], df2['CO: 4'] = df2['CO: 3'].astype(int), df2['CO: 4'].astype(int)
df2['CO: 5'], df2['CO: 6'] = df2['CO: 5'].astype(int), df2['CO: 6'].astype(int)

X2, y3, y4, y5, y6 = df2.iloc[:, 1:-4], df2.iloc[:, -4], df2.iloc[:, -3], df2.iloc[:, -2], df2.iloc[:, -1]

# Train test split

X_train3, X_test3, y_train3, y_test3 = train_test_split(X2, y3, test_size = 0.15, random_state = 30)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X2, y4, test_size = 0.15, random_state = 30)
X_train5, X_test5, y_train5, y_test5 = train_test_split(X2, y5, test_size = 0.15, random_state = 30)
X_train6, X_test6, y_train6, y_test6 = train_test_split(X2, y6, test_size = 0.15, random_state = 30)

In [43]:
#Random forest based feature selection for descriptor1
model_RF1 = RandomForestClassifier(n_estimators = 100, n_jobs=-1, random_state=42)
model_RF1.fit(X_train1, y_train1)

features1 = X.columns
importances1 = model_RF1.feature_importances_
indices1 = np.argsort(importances1)
#Random forest based feature selection for descriptor2
model_RF2 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
model_RF2.fit(X_train2, y_train2)

features2 = X.columns
importances2 = model_RF2.feature_importances_
indices2 = np.argsort(importances2)
#Random forest based feature selection for descriptor3
model_RF3 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
model_RF3.fit(X_train3, y_train3)

features3 = X2.columns
importances3 = model_RF3.feature_importances_
indices3 = np.argsort(importances3)
#Random forest based feature selection for descriptor4
model_RF4 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
model_RF4.fit(X_train4, y_train4)

features4 = X2.columns
importances4 = model_RF4.feature_importances_
indices4 = np.argsort(importances4)
#Random forest based feature selection for descriptor5
model_RF5 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
model_RF5.fit(X_train5, y_train5)

features5 = X2.columns
importances5 = model_RF5.feature_importances_
indices5 = np.argsort(importances5)
#Random forest based feature selection for descriptor6
model_RF6 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
model_RF6.fit(X_train6, y_train6)

features6 = X2.columns
importances6 = model_RF6.feature_importances_
indices6 = np.argsort(importances6)

In [44]:
list1 = list(reversed([features1[i] for i in indices1[-709:]]))
list2 = list(reversed([features2[i] for i in indices2[-860:]]))
list3 = list(reversed([features3[i] for i in indices3[-1875:]]))
list4 = list(reversed([features4[i] for i in indices4[-1497:]]))
list5 = list(reversed([features5[i] for i in indices5[-1944:]]))
list6 = list(reversed([features6[i] for i in indices6[-2254:]]))

# MODEL BUILDING

In [45]:
train_data_1 = pd.read_csv('Dataset_1_Training.csv', index_col=0).T
train_data_2 = pd.read_csv('Dataset_2_Training.csv', index_col=0).T
test_data_1 = pd.read_csv('Dataset_1_Testing.csv', index_col=0).T
test_data_2 = pd.read_csv('Dataset_2_Testing.csv', index_col=0).T

X_train_1 = train_data_1.drop(columns=['CO: 1','CO: 2'])
Y_train_CO1 = train_data_1['CO: 1']
Y_train_CO2 = train_data_1['CO: 2']

X_train_2 = train_data_2.drop(columns=['CO: 3','CO: 4','CO: 5','CO: 6'])
Y_train_CO3 = train_data_2['CO: 3']
Y_train_CO4 = train_data_2['CO: 4']
Y_train_CO5 = train_data_2['CO: 5']
Y_train_CO6 = train_data_2['CO: 6']


In [46]:
def feature_selection(train_df, gene_list):
    '''return X_train_CO'''
    train_co = train_df.drop(columns=[col for col in train_df.columns.values if col not in gene_list])
    return train_co

X_train_CO1 = feature_selection(X_train_1, list1)
X_train_CO2 = feature_selection(X_train_1, list2)
X_train_CO3 = feature_selection(X_train_2, list3)
X_train_CO4 = feature_selection(X_train_2, list4)
X_train_CO5 = feature_selection(X_train_2, list5)
X_train_CO6 = feature_selection(X_train_2, list6)

X_test_CO1 = feature_selection(test_data_1,list1)
X_test_CO2 = feature_selection(test_data_1,list2)
X_test_CO3 = feature_selection(test_data_2,list3)
X_test_CO4 = feature_selection(test_data_2,list4)
X_test_CO5 = feature_selection(test_data_2,list5)
X_test_CO6 = feature_selection(test_data_2,list6)


In [47]:
#final classification model

cl2 = LogisticRegression()
cl1 = AdaBoostClassifier(n_estimators=10)
cl3 = KNeighborsClassifier(n_neighbors=10)
#c4 = KNeighborsClassifier(n_neighbors=3)
en_1 = make_pipeline(StandardScaler(), VotingClassifier(estimators = [('cl1', cl1),('cl2', cl2),('cl3', cl3)]))
en_1.fit(X_train_CO1, Y_train_CO1)
y_pred_CO1 = en_1.predict(X_test_CO1)
y_pred_train_CO1 = en_1.predict(X_train_CO1)

cl2 = LogisticRegression()
cl1 = AdaBoostClassifier(n_estimators=25)
cl3 = KNeighborsClassifier(n_neighbors=10)
#c4 = RandomForestClassifier(max_depth=10)
en_2 = make_pipeline(StandardScaler(), PCA(0.95), VotingClassifier(estimators = [('cl1', cl1),('cl2', cl2),('cl3', cl3)]))
en_2.fit(X_train_1, Y_train_CO2)
y_pred_CO2 = en_2.predict(test_data_1)
y_pred_train_CO2 = en_2.predict(X_train_1)

cl1 = SVC(gamma='auto', probability=True)
cl2 = BaggingClassifier(LogisticRegression(), max_samples = 0.5, max_features=0.5)
cl3 = RandomForestClassifier(n_estimators=5)
#c4 = KNeighborsClassifier(n_neighbors=50)
en_3 = make_pipeline(StandardScaler(), VotingClassifier(estimators = [('cl1', cl1),('cl2', cl2),('cl3', cl3)]))
en_3.fit(X_train_CO3, Y_train_CO3)
y_pred_CO3 = en_3.predict(X_test_CO3)
y_pred_train_CO3 = en_3.predict(X_train_CO3)

cl1 = BaggingClassifier(KNeighborsClassifier(), max_samples = 0.5, max_features=0.5)
cl2 = AdaBoostClassifier(n_estimators=50)
cl3 = RandomForestClassifier(n_estimators=5)
en_4 = make_pipeline(StandardScaler(), VotingClassifier(estimators = [('cl1', cl1),('cl2', cl2),('cl3', cl3)]))
en_4.fit(X_train_2, Y_train_CO4)
y_pred_CO4 = en_4.predict(test_data_2)
y_pred_train_CO4 = en_4.predict(X_train_2)

en_5 = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=20))
en_5.fit(X_train_CO5, Y_train_CO5)
y_pred_CO5 = en_5.predict(X_test_CO5)
y_pred_train_CO5 = en_5.predict(X_train_CO5)

#c4 = BaggingClassifier(LogisticRegression(), max_samples = 0.5, max_features=0.5)
cl1 = GradientBoostingClassifier(n_estimators=100)
cl2 = RandomForestClassifier(n_estimators=10)
cl3 = RandomForestClassifier(n_estimators=100)
en_6 = make_pipeline(StandardScaler(), PCA(0.95), VotingClassifier(estimators = [('cl1', cl1),('cl2', cl2),('cl3', cl3)]))
en_6.fit(X_train_2, Y_train_CO6)
y_pred_CO6 = en_6.predict(test_data_2)
y_pred_train_CO6 = en_6.predict(X_train_2)

C:\Users\shash\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
result = np.concatenate((y_pred_CO1,y_pred_CO2,y_pred_CO3,y_pred_CO4,y_pred_CO5,y_pred_CO6), axis=0)
answer = pd.DataFrame([int(i) for i in range(len(result))], columns = ['Id'])
answer['Predicted'] = pd.DataFrame(result)
answer['Predicted'] = answer['Predicted'].astype(int)

# Write CSV file
answer.to_csv('BE18B006_BE18B011_result.csv', encoding='utf-8', mode='a', header=True, index=False)